# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import sys

%matplotlib inline

In [2]:
sys.path.append('/home/kevinzhao/Udacity/data/')

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_message_categories.db')
df = pd.read_sql_table('message_categories', engine)
# df.dropna(how='any', inplace=True)
X = df['message']
y = df.iloc[:,4:]
categories = y.columns

In [5]:
df[~(df.isnull().any(axis=1))|(df.original.isnull())].shape, df[~(df.isnull().any(axis=1))].shape

((26216, 40), (10170, 40))

In [6]:
df = df[~(df.isnull().any(axis=1))|((df.original.isnull())&~(df.offer.isnull()))]

### 2. Write a tokenization function to process your text data

In [4]:
import nltk
import re

In [5]:
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

In [6]:
def tokenize(text):
    
    # get tokens from text
    tokens = WhitespaceTokenizer().tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # clean tokens
    processed_tokens = []
    for token in tokens:
        token = lemmatizer.lemmatize(token).lower().strip('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
        token = re.sub(r'\[[^.,;:]]*\]', '', token)
        
        # add token to compiled list if not empty
        if token != '':
            processed_tokens.append(token)
        
    return processed_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [11]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
import random
from sklearn.model_selection import train_test_split

In [9]:
random.seed(20181022)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [30]:
pipeline.fit(X_train, y_train)

/Users/kevinzhao/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [23]:
from sklearn.metrics import classification_report, accuracy_score

In [32]:
y_pred = pipeline.predict(X_test)

/Users/kevinzhao/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [33]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=categories))

                        precision    recall  f1-score   support

               request       0.85      0.36      0.50       921
                 offer       0.00      0.00      0.00        30
           aid_related       0.74      0.55      0.63      2174
          medical_help       0.61      0.08      0.14       423
      medical_products       0.59      0.05      0.09       278
     search_and_rescue       0.88      0.10      0.18       137
              security       0.00      0.00      0.00        85
              military       0.38      0.03      0.06       175
           child_alone       0.00      0.00      0.00         0
                 water       0.79      0.28      0.41       327
                  food       0.81      0.32      0.46       582
               shelter       0.73      0.18      0.28       456
              clothing       0.67      0.09      0.16        88
                 money       0.62      0.04      0.07       131
        missing_people       0.33      

/Users/kevinzhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/kevinzhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'vect__ngram_range':[(1,2),(2,2)],
            'clf__estimator__n_estimators':[50, 100]
             }

cv = GridSearchCV(pipeline, parameters)

In [ ]:
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=categories))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer

In [14]:
def compute_text_length(data):
    return np.array([len(text) for text in data]).reshape(-1, 1)

In [18]:
pipeline = Pipeline([
    ('features', FeatureUnion([('text', Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                                                 ('tfidf', TfidfTransformer()),
                                                 ])),
                              ('length', Pipeline([('count', FunctionTransformer(compute_text_length, validate=False))]))]
                             )),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

In [19]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

KeyboardInterrupt: 

In [ ]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=categories))

In [22]:
parameters = {'features__text__vect__ngram_range':[(1,2),(2,2)],
            'clf__estimator__n_estimators':[50, 100]
             }

In [23]:
cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
y_pred = cv.predict(X_test)

In [25]:
print(accuracy_score(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred])))

0.7412280701754386


In [22]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=categories))

                        precision    recall  f1-score   support

               related       0.98      0.82      0.89       906
               request       1.00      0.52      0.69        23
                 offer       0.96      0.88      0.92      2113
           aid_related       0.99      0.69      0.81       400
          medical_help       0.99      0.75      0.85       248
      medical_products       1.00      0.69      0.81       147
     search_and_rescue       1.00      0.61      0.75        94
              security       0.99      0.73      0.84       179
              military       0.00      0.00      0.00         0
           child_alone       0.98      0.84      0.91       318
                 water       0.97      0.87      0.92       543
                  food       0.97      0.75      0.85       452
               shelter       0.98      0.72      0.83        85
              clothing       1.00      0.62      0.76       123
                 money       1.00      

/Users/kevinzhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/Users/kevinzhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/kevinzhao/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [36]:
import pickle

In [ ]:
pickle.dump(cv, open('model.p', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.